# RARR Edits
- Utility script to study edits done by RARR

In [ ]:
import json
from src.utils.get_sentences import get_sentences
from src.experiments.load_experiment import load_experiment_df, Experiment
from src.column import Column
from src.models.citation import SentenceWithCitations

df, _ = load_experiment_df(Experiment.RARR_LLAMA_8B)
df_base, _ = load_experiment_df(Experiment.BASE_LLAMA_8B)

total_sentences = 0
total_edits = 0
for i, row in df.iterrows():
    try:

        row_base = df_base.iloc[i]
        ga = row_base[Column.GENERATED_ANSWER]
        sentences = get_sentences(ga)

        sentences_rarr = row[Column.GENERATED_CITATIONS]
        sentences_rarr = json.loads(sentences_rarr)
        sentences_rarr = [SentenceWithCitations.model_validate(x) for x in sentences_rarr]
        sentences_rarr = [x.sentence for x in sentences_rarr]

        is_edited = False

        for s, sr in zip(sentences, sentences_rarr):
            total_sentences += 1
            if s != sr:
                is_edited = True
                total_edits += 1
                print()
                print("Sentence:", s)
                print("Edit:", sr)

        if is_edited:
            print("Question:", row[Column.QUESTION])
            print("Base answer:", " ".join(sentences))
            print("Rarr answer:", " ".join(sentences_rarr))
            print(" --- ")
        
    except Exception as e:
        print(i, e)

print(total_edits / total_sentences)
print(total_edits)
print(total_sentences)